In [7]:
from sqlalchemy import create_engine, text
import pandas as pd
from pathlib import Path
import re
from typing import Set, Dict, List, Tuple
import os, uuid

from sqlalchemy.orm.base import PASSIVE_OFF

In [8]:
USER = "postgres"
HOST = "localhost"
PORT = "5432"
PASSWORD = "user"


In [9]:
DB_ADMIN_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/postgres"
engine_admin = create_engine(DB_ADMIN_URL, isolation_level="AUTOCOMMIT")

In [10]:
with engine_admin.connect() as conn:
    conn.execute(text("CREATE DATABASE synthea"))
print("Database 'synthea' creato (o già esistente)")

ProgrammingError: (psycopg2.errors.DuplicateDatabase) database "synthea" already exists

[SQL: CREATE DATABASE synthea]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [11]:
DB_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/synthea"
engine = create_engine(DB_URL)
print("Connesso al database synthea")

Connesso al database synthea


In [12]:
schema = open("sql/importDB.sql").read()

with engine.begin() as conn:
    if schema.strip():
        conn.execute(text(schema))
        print("Schema creato")
    else:
        print("Errore")

Schema creato


Comando per scaricare 1000 pazienti:
java -jar synthea-with-dependencies.jar -p 1000

In [14]:
# 1) Imposta il percorso alla cartella CSV di Synthea

CSV_BASE = Path("/Users/sarabuzzi/synthea/syntheaProject/output/csv")

CSV_BASE.resolve(), CSV_BASE.exists()


(PosixPath('/Users/sarabuzzi/synthea/syntheaProject/output/csv'), True)

In [15]:

def load_csv_to_sql(table, file):
    df = pd.read_csv(file)
    df.columns = [c.lower() for c in df.columns]
    df.to_sql(table, engine, if_exists="append", index=False)


load_csv_to_sql("patients", f"{CSV_BASE}/patients.csv")
load_csv_to_sql("encounters", f"{CSV_BASE}/encounters.csv")
load_csv_to_sql("conditions", f"{CSV_BASE}/conditions.csv")
load_csv_to_sql("medications", f"{CSV_BASE}/medications.csv")
load_csv_to_sql("observations", f"{CSV_BASE}/observations.csv")


In [11]:
def run(sql_or_text, show=False):
    with engine.begin() as conn:
        stmt = text(sql_or_text) if isinstance(sql_or_text, str) else sql_or_text
        result = conn.execute(stmt)
        if result.returns_rows:
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
            if show:
                display(df)
            return df
        return None

In [12]:
# test query
run("""
    SELECT BIRTHDATE
    FROM PATIENTS
    WHERE Id = 'c10e497b-ed25-d6c6-e043-144724bf84da'
    """, show=True);


,birthdate
0,2009-01-23


FRAMMENTAZIONE VERTICALE

# PATIENTS
Owner(PATIENTS) = { id, deathdate, first, last, ssn, drivers, passport, address, city, state, county, fips, zip, lat, lon, income, birthplace }
Server(PATIENTS) = { id, birthdate, gender, race, ethnicity, marital }



In [114]:
sql = open("sql/fragmentPatients.sql").read()

with engine.begin() as conn:
    if sql.strip():
        conn.execute(text(sql))
        print("Frammentazione creata")
    else:
        print("Errore")

# O semplicemente uso la funzione run(sql)

Frammentazione creata
